![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20RoBERTa.ipynb)

## Import RoBERTa models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for RoBERTa from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.


In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [roberta-base](https://huggingface.co/roberta-base) model from HuggingFace as an example
- In addition to `TFRobertaModel` we also need to save the `RobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf

MODEL_NAME = 'roberta-base'

# let's keep the tokenizer variable, we need it later
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
# let's save the tokenizer
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFRobertaModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFRobertaModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


INFO:tensorflow:Assets written to: ./roberta-base/saved_model/1/assets


INFO:tensorflow:Assets written to: ./roberta-base/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 974328
-rw-r--r--  1 maziyar  staff        638 Dec 15 17:27 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 15 17:27 saved_model
-rw-r--r--  1 maziyar  staff  498849472 Dec 15 17:27 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 18032
drwxr-xr-x  2 maziyar  staff       64 Dec 15 17:27 assets
-rw-r--r--  1 maziyar  staff       56 Dec 15 17:27 fingerprint.pb
-rw-r--r--  1 maziyar  staff   165513 Dec 15 17:27 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  9057879 Dec 15 17:27 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 15 17:27 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 2864
-rw-r--r--  1 maziyar  staff  456318 Dec 15 17:26 merges.txt
-rw-r--r--  1 maziyar  staff     957 Dec 15 17:26 special_tokens_map.json
-rw-r--r--  1 maziyar  staff    1342 Dec 15 17:26 tokenizer_config.json
-rw-r--r--  1 maziyar  staff  999355 Dec 15 17:26 vocab.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.json` and `merges.txt` files from the tokenizer
- all we need is to first convert `vocab.json` to `vocab.txt` and copy both `vocab.txt` and `merges.txt` into `saved_model/1/assets` which Spark NLP will look for

In [ ]:

# let's make sure we sort the vocabs based on their ids first
vocabs = tokenizer.get_vocab()
vocabs = sorted(vocabs, key=vocabs.get)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in vocabs:
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/merges.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save RoBERTa in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.4
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.4


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `RoBertaEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively..


In [ ]:
from sparknlp.annotator import *

roberta = RoBertaEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('roberta_base') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
roberta.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your RoBERTa model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 991336
drwxr-xr-x  5 maziyar  staff        160 Dec 15 17:27 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 15 17:27 metadata
-rw-r--r--  1 maziyar  staff  507563632 Dec 15 17:27 roberta_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊 

In [ ]:
roberta_loaded = RoBertaEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(True)

In [ ]:
roberta_loaded.getStorageRef()

'roberta_base'

That's it! You can now go wild and use hundreds of RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀 
